# [Experimental, OTel] Compute Direct Feedbacks and Output as Spans

Prerequisites:
- That you use Snowflake (and Snowpark)
- that you have events (OTel spans)
- that you have a table whose schema is compatible with the TruLens-defined Event ORM ([see orm.py](../../../src/core/trulens/core/database/orm.py#L409))
- that your database is SQLAlchemy-compatible (we use the function `get_events()`, which is [defined in sqlalchemy.py](../../../src/core/trulens/core/database/sqlalchemy.py#L1640))


Alternatively, you may BYO table, as long as we can fetch from it and write to it!
- You are welcome and encouraged to contribute new connectors and their corresponding access methods as well via PR!

In [ ]:
from collections import defaultdict
import json
import os
from typing import Dict, List, Optional

import pandas as pd
from snowflake.snowpark import Session
from trulens.apps.app import TruApp
from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core.feedback import Feedback
from trulens.core.feedback.selector import Selector
from trulens.core.feedback.selector import Trace
from trulens.core.session import TruSession
from trulens.feedback.computer import compute_feedback_by_span_group

## Setup

In [ ]:
# Environment variables.
os.environ["TRULENS_OTEL_TRACING"] = "1"

# For ease of access.
app_name = "REPLACE_APP_NAME"
app_version = "REPLACE_APP_VERSION"

In [ ]:
# Connect to Snowflake via Snowpark.
connection_params: Dict[str, str] = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    # "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
    "authenticator": "externalbrowser",
    "database": os.environ["SNOWFLAKE_DATABASE"],
    "schema": os.environ["SNOWFLAKE_SCHEMA"],
    "role": os.environ["SNOWFLAKE_ROLE"],
    "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
}
snowpark_session = Session.builder.configs(connection_params).create()

In [ ]:
# Create TruSession for Snowflake.
connector = SnowflakeConnector(
    snowpark_session=snowpark_session,
    use_account_event_table=False,
)
tru_session = TruSession(connector=connector)

## Define and run evals/feedback functions

In [ ]:
# Create custom feedback.
def uses_anthropic(trace: Trace) -> float:
    if trace.events is None:
        return 0.0
    if any(
        trace.events["processed_content"].apply(
            lambda curr: "anthropic" in str(curr)
        )
    ):
        return 1.0
    return 0.0

In [ ]:
# Create custom feedback function.
f_uses_anthropic = Feedback(
    uses_anthropic,
    name="Uses Anthropic",
    description="Whether the model uses Anthropic.",
).on({
    "trace": Selector(
        trace_level=True,
        span_attribute="ai.observability.agent.tool.cortex_analyst.model_name",
    )
})

all_feedbacks = [f_uses_anthropic]

In [ ]:
# Create fake app to use with TruSession (to satisfy TruApp requirement).
class FakeApp:
    pass


fake_app = FakeApp()
tru_app = TruApp(
    fake_app,
    app_name=app_name,
    app_version=app_version,
    feedbacks=all_feedbacks,
)

# Make sure no evaluation threads are running, and reset threads for future compute runs.
tru_app.stop_evaluator()

tru_app.compute_feedbacks(raise_error_on_no_feedbacks_computed=False)
print("DONE")

## Experimental

Below, we are trying to exploring methods to reduce the amount of setup required in order to get a user to a specific evaluation/feedback computation by circumventing certain abstractions (in this case, `TruApp`).

TODOs:
- verify that this also exports feedback spans to the table
- verify if this shows up in TruLens Streamlit UI

In [ ]:
# (Experimental) Directly compute feedbacks without TruApp (to simplify path to feedback computation as much as possible)
# Note: the logic here is identical to the TruApp.compute_feedbacks method
def directly_compute_feedbacks_for_events(
    events: pd.DataFrame,
    feedbacks: List[Feedback],
    app_name: Optional[str] = None,
    app_version: Optional[str] = None,
) -> None:
    if events is None:
        if app_name is None and app_version is None:
            raise ValueError(
                "Either events must be provided or both app_name and app_version must be provided"
            )
        # Get all events associated with a provided app name and version.
        events = connector.get_events(
            app_name=app_name, app_version=app_version
        )
    for feedback in all_feedbacks:
        compute_feedback_by_span_group(
            events,
            feedback.name,
            feedback.imp,
            feedback.higher_is_better,
            feedback.selectors,
            feedback.aggregator,
            raise_error_on_no_feedbacks_computed=False,
        )

In [ ]:
# Run feedback computation.
directly_compute_feedbacks_for_events(
    # events=None,
    feedbacks=all_feedbacks,
    app_name=app_name,
    app_version=app_version,
)

In [ ]:
## TODO: verify that this works.

## Testing

In [ ]:
import pandas as pd

# Visualize Event table results from a CSV file.
# NOTE: You can also download this CSV from the Snowsight UI.
df = pd.read_csv("REPLACE_PATH_TO_EVENT_TABLE_CSV")
record_attributes = df["RECORD_ATTRIBUTES"]
record_ids = set()
for curr in record_attributes:
    curr = json.loads(curr)
    if "ai.observability.record_id" in curr:
        record_ids.add(curr["ai.observability.record_id"])
for record_id in record_ids:
    attribute_count = defaultdict(list)
    for attributes in record_attributes:
        attributes = json.loads(attributes)
        if attributes.get("ai.observability.record_id") != record_id:
            continue
        for k, v in attributes.items():
            attribute_count[k].append(v)
    print("RECORD_ID:", record_id)
    for k, v in attribute_count.items():
        if len(v) == 1:
            print(f"{k}: {v}")
    print()